# NGS Workflows

A typical NGS experiment involves more than one sample, potential 10's or 100's of samples. During the experiment, a sample may be split across multiple libraries and and a library may be split across multiple sequencing runs (lanes). For example, you may have to increase the number of runs for a specific sample to increase the read-depth (sequencing volume), so you have to prepare multiple libraries.

Therefore you need a coordinated workflow, driven by standard software to bring it reliably together. Read alignment is just the first part of that. Once you have a BAM file for each sequencing run you need to merge them together to produce a BAM file for the library. At this stage it is important to perform de-duplication on the merged data. The main purpose of removing duplicates is to mitigate the effects of PCR amplification bias introduced during library construction. PCR duplicates erroneously inflate the coverage and, if not removed, can give the illusion of high confidence when it is not really there which can have an effect on downstream analysis such as variant calling.

The figure below outlines a typical NGS workflow:

![Typical NGS Workflow](images/workflow.png "Typical NGS Workflow")

In this part of the tutotial, we have two lanes of illumina sequencing data produced from a single library of yeast. We will use the BWA aligner to align the data to the Saccromyces cerevisiae genome
(ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna/) and produce a merged BAM file for the library. 

To begin go to the following directory:

In [ ]:
cd /home/manager/course_data/read_alignment/data/Exercise2/60A_Sc_DBVPG6044/library1

## Index the reference

In [ ]:
bwa index ../../../../ref/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa.gz

## Align the first sequencing run

Recall that to align a lane of data to a reference genome we must perform the following steps:

* Align the data
* Convert from SAM to BAM
* Sort the BAM file
* Index the sorted BAM file

### Find the data

Go to the directory that contains the data for the first sequencing run:

In [ ]:
cd lane1

### Run the alignment
Remember from earlier in the tutorial that the Unix pipe command allows you to feed the output of one command into the next command. So using Unix pipes, we can combine all of the alignment steps together into one command and do all of this data processing together and avoid writing intermediate files. To do this type the command:

In [ ]:
bwa mem -M -R '@RG\tID:lane1\tSM:60A_Sc_DBVPG6044' ../../../../ref/Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa.gz s_7_1.fastq.gz s_7_2.fastq.gz | samtools view -bS - | samtools sort -T temp -O bam -o lane1.sorted.bam -

__Q1: What do the -M and -R options do?__

__Q2: What does the -bS option do?__

Now index the BAM file:

In [ ]:
samtools index lane1.sorted.bam

### Generate QC stats

Now use samtools to collect some statistics and generate QC plots from the alignment in the BAM file. Type the commands:

In [ ]:
samtools stats lane1.sorted.bam > lane1.stats.txt

In [ ]:
plot-bamstats -p plot/ lane1.stats.txt

Now look at the output and answer the following questions:

**Q3:** What is the total number of reads?

**Q4:** What proportion of the reads were mapped?

**Q5:** How many reads were paired correctly/properly?

**Q6:** How many reads mapped to a different chromosome?

**Q7:** What is the insert size mean and standard deviation?

In a web browser open the file called plots.html to view the QC information.

**Q8:** How many reads have zero mapping quality?

**Q9:** Which of the first fragments or second fragments are higher base quality on average? 

## Align the second sequencing run

There is a second lane of sequencing data in the `library1` directory contained in the directory `lane2`. We want to also align this sequncing data and produce a BAM file.

Go to the directory that contains the data for the second sequencing run:

In [ ]:
cd ../lane2

Now align the data in this directory to the yeast reference genome and produce a sorted BAM file.

**Note:** This time when you use the `bwa mem` command use the following header option to specify lane2 as the read
group ID:

`@RG\tID:lane2\tSM:60A_Sc_DBVPG6044`

**Q10:** What is the size of the BAM file that is produced? 

## Merge the BAM files

Go to the directory that contains the data for the library `60A_Sc_DBVPG6044/library1` . Use `ls` to get a listing of the files and directories contained in this directory.

In [ ]:
cd ..

In [ ]:
pwd

In [ ]:
ls

You will notice that there are two directories called `lane1` and `lane2`. There were two sequencing lanes produced from this sequencing library. In order to mark library PCR duplicates, we need to merge the two lane BAM files together to produce a single BAM file. We will use the picard tool called ‘MergeSamFiles’ (http://picard.sourceforge.net) to merge the lane BAM files. 

To find the options for ‘MergeSamFiles’ command, type:

In [ ]:
picard MergeSamFiles

Now use the `I=` option to specify both the input BAM files and the `O=` option to specify the
output file (e.g. library1.bam). **Note: Multiple input files can be specified using multiple `I=` options**

## Mark PCR duplicates

We will use a program called ‘MarkDuplicates’ that is part of Picard tools (http://picard.sourceforge.net) to remove PCR duplicates that may have been introduced during the library construction stage. To find the options for ‘MarkDuplicates’ type:

In [ ]:
picard MarkDuplicates

Now use the `I=` option to specify the input BAM file and the `O=` option to specify the output file (e.g. library1.markdup.bam). You will also need to specify the duplication metrics output file using `M=` (e.g. library1.markdup.metrics). 

**Don’t forget to index your final bam file using `samtools index`.

**Q11:** From looking at the output metrics file - how many reads were marked as duplicates? 

**Q12:** What was the percent duplication?

## Visualise the alignment

Go to the directory containing the reference genome and uncompress the file as IGV cannot read a compressed file.

In [ ]:
cd /home/manager/course_data/read_alignment/data/ref

In [ ]:
gunzip Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa.gz

Start IGV by typing:

In [ ]:
igv &

### Load the reference genome

**On the top menu bar go to ' _Genomes –> Load Genome From File..._ ', go to the "ref" directory and select the file "Saccharomyces_cerevisiae.R64-1-1.dna.toplevel.fa" and click ' _Open_ '.**

![Loading the yeast reference genome](images/load-yeast-ref.png "Loading the yeast reference genome")

![Loading the yeast reference genome from a file](images/yeast-ref-dialog.png "Loading the yeast reference genome from a file")

### Load the alignment

**To load the merged BAM file, on the top menu bar go to ' _File –> Load from File..._ ' and select the library BAM file that you created in the previous step.**

![Loading the yeast alignment"](images/load-yeast-bam.png "Loading the yeast alignment")

![Loading the yeast alignment](images/yeast-bam-dialog.png "Loading the yeast alignment")

## Exercises

1. Go to Chromosome IV and position 764,292. (**Hint: use the navigation bar across the top**)

2. What is the reference base at this position?

3. Do the reads agree with the reference base?

4. What about the adjacent position (IV:764,293)? What is the reference base at this position? Is it supported by the reads?

5. Go to Chromosome IV and position 766,589.

6. What sort of mutation are the alignments indicating might be present?

7. Go to Chromosome IV and position 770,137 using the navigation bar across the top.

8. What sort of mutation are the alignments indicating might be present? Is there anything in the flanking sequence of the reference genome that might make you suspicious about this mutation?

9. Convert the BAM file to a CRAM file

You have reached the end of the Read Alignment tutorial.